In [43]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import expon
from scipy.stats import uniform
from scipy.integrate import quad

In [3]:
def FOC(kj,ki,gammai,Vj,t,r,p):

    # The 'x' vector to be optimized is the vector of physician strategies, kj

    # Patient strategies
    Ui_aux = Vj*ki.reshape((-1,1)) - t     # Vj x ki
    ki_greater = np.array(ki.reshape((-1,1)) >= kj.reshape((1,-1))).astype(int)  # I x J boolean mask of ki > kj
    Ui = Ui_aux + ki_greater * gammai.reshape((-1,1))   # Vj x ki + gammai if ki > kj
    exp_Ui = np.exp(Ui)     # exp(Ui)
    αi = np.zeros((I,J))
    αi[Ui > 0] = exp_Ui[Ui > 0]
    sum_αi = np.sum(αi, axis = 1).reshape((-1,1))
    Si = np.divide(αi, sum_αi, where=sum_αi!=0, out=np.zeros_like(αi))   # I x J vector of patient strategies (logit)

    # Aggregates
    Qi = np.sum(Si, axis = 0)
    Xi = np.sum(ki_greater * Si, axis = 0)

    return r - p * Xi


In [77]:
# Number of doctors and patients
I = 100
J = 50

# Doctor parameters
Vj = np.random.random(J)
r =0.1
p = 100000*np.random.random(J)
t= 0.2

# Patient parameters
ki  = np.random.random(I)
gammai  = np.random.random(I)

# Doctor strategies guess
kj  = 0.5*np.ones(J)

In [368]:
def newton(f, Df, x_0, tol=1e-7, max_iter=100_000):
    x = x_0

    # Implement the zero-finding formula
    def q(x):
        Df_x = Df(x)  # Compute Df(x) once to avoid redundant calculations
        return np.where(Df_x == 0, 0, x - f(x) / Df_x)

    error = tol + 1
    n = 0
    while np.any(error > tol):
        n += 1
        if(n > max_iter):
            break
        y = q(x)
        error = np.where(y == 0, 0, np.abs(x - y))
        
        x = y
        x[x<0] = 0
        if all(x == 0):
            break
    return x, error


h = 0.001
f = lambda x: FOC(x, ki,gammai,Vj,t,r,p)
Df = lambda x: (f(x + h) - f(x))*(1/h)
k_guess = 0.5*np.ones(J)

a, b = newton(f, Df, k_guess, tol=1e-3, max_iter=1)

In [367]:
Df(a)

array([      0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,

array([     0.        ,      0.        ,      0.        ,      0.        ,
       198859.98855163])

In [282]:
a

array([0., 0., 0., 0., 0.])

In [81]:
def uij(ki,gammai,Vj,t,kj):
            "Utility patient i receives from doctor j"

            return Vj * ki - t + np.where(ki >= kj, gammai, 0)
        
def aij(u,λ):
            "Intermediate function to calculate sij"

            return np.where(u > 0, np.exp(λ*u), 0)
                
def sij(ki,gammai,Vj,t,kj,λ):    #It takes the J-sized vectors of all doctors' Vj and κj as arguments
            "Probability that patient i visits doctor j"

            u = uij(ki,gammai,Vj,t,kj)

            return aij(u,λ)/np.sum(aij(u,λ))

ki = 0.1*np.ones((1,2))
gammai = 0.5
λ = 1
t = 0
Vj = Vj.reshape((-1,1))
kj = kj.reshape((-1,1))

b = 5

F = expon(scale=1/b)
G = uniform
H = uniform(scale = 2)

f = lambda x: F.pdf(x)
g = lambda x: G.pdf(x)


s_k = lambda x: sij(x,gammai,Vj,t,kj,λ)

l = kj.reshape(1,-1)

s_gamma = lambda x: np.diagonal(sij(kj.reshape(1,-1),x,Vj,t,kj,λ))

gamma_integrand = lambda x: s_gamma(x) * g(x)

n = 101

x = np.linspace(0,1,n)
# gamma_integrand(x)

# Initialize a list to store results
results = []

# Loop over each value in x_values
for i in x:
    result = gamma_integrand(i)
    results.append(result)

# Convert results to a NumPy array for easier manipulation
results = np.array(results)

sum = np.sum(results, axis = 0)/n
# DIVIDO POR N O NO? NO SE

sum





array([0.00039528, 0.00043657, 0.00036302, 0.00042512, 0.00044957,
       0.00048414, 0.00046034, 0.00050636, 0.00035242, 0.00038092,
       0.00048073, 0.00036154, 0.00039855, 0.00033676, 0.0003425 ,
       0.00039206, 0.00036748, 0.00047674, 0.00039302, 0.00039899,
       0.00040057, 0.00032216, 0.00040179, 0.00039077, 0.00045464,
       0.00046471, 0.00034728, 0.00042756, 0.00031407, 0.0003466 ,
       0.00044067, 0.00038551, 0.00036842, 0.00037743, 0.00045591,
       0.00041276, 0.00048274, 0.00040845, 0.00033107, 0.00041391,
       0.00031935, 0.00048726, 0.00033959, 0.00039539, 0.00035249,
       0.00034056, 0.00043562, 0.00042866, 0.00031956, 0.00043238])